## AdaBoost

[AdaBoost](https://en.wikipedia.org/wiki/AdaBoost) wiki 上有很好的推导过程。

AdaBoost(Adaptive Boost), 自适应 Boost, 即当前迭代的分类器会根据之前的分类器做自适应的调整。

AdaBoost, 对噪声样本和 Outlier 敏感，但不容易过拟合。

### 概览

#### 训练

AdaBoost 是训练 boost 分类器的一个特殊方法。一个 boost 分类器其形式为：

$$
F_T(x) = \sum_{t=1}^{T} f_t(x)
$$

其中 $f_t$ 是弱学习器(weak learner), 返回样本的类别。

每一个弱学习器(weak learner) 都生成一个假设函数 $h(x_i)$。在 t 迭代时，当前 t 迭代对应的弱分类器加上一个参数 $\alpha_t$, 则训练误差为：

$$
E_t = \sum_{i}E[F_{t - 1}(x_i) + \alpha_t h(x_i)]
$$

$F_{t - 1}(x_i)$ 是目前为止已经生成的 boosted 分类器。

#### 权重

在训练的过程中，第一个样本都会被赋予一个权重 $w_{i, t}$, 该权重等于 $E(F_{t-1}(x_i))$。即

$$
w_{i, t} = E\big(F_{t-1}(x_i)\big)
$$


### 推导

假设，有数据集 $\{(x_1, y_1), \dots, (x_N, y_N)\}, y_i \in \{-1, 1\}$, 和一组弱分类器 $\{k_1, \dots, k_L \}, k_j(x_i) \in \{-1, 1\}$.

则在 $m - 1$ 迭代后，我们 boosted 分类器的线性组合形式为：

$$
C_{(m - 1)}(x_i) = \alpha_1 k_1 (x_i) + \cdots + \alpha_{m-1} k_{m-1} (x_i)
$$

在 $m$ 次迭代时，我们得到一个更优的分类器。

$$
C_{m}(x_i) = C_{(m - 1)} + \alpha_m k_m(x_i)
$$


所以问题在于：对于弱分类器 $k_m 怎样选择其参数 \alpha_m $。

我们使用[指数损失函数](https://en.wikipedia.org/wiki/Loss_functions_for_classification#Exponential_loss) 来定义分类器 $C_m$  的误差：

$$
E = \sum_{i} e^{- y_i C_m(x_i)} = \sum_{i = 1}^{N} e^{-y_iC_{m - 1}(x_i)} e^{-y_i \alpha_m k_m(x_i)}
$$

令 $w_i^{(m)} =  e^{-y_iC_{m - 1}(x_i)}, m > 1. 当 m = 1 时， w_i^{(1)} = 1 $。则上式可以写为：

$$
E = \sum_{i = 1}^{N} w_i^{(m)} e^{-y_i \alpha_m k_m(x_i)}
$$

因为 $y_i k_m(x_i)$ 要么等于 $1$ 要么等于 $ -1 $, 所以上式又可以写为：

\begin{align}
E &= \sum_{i = 1}^{N} w_i^{(m)} e^{-y_i \alpha_m k_m(x_i)} 
\\
&= \sum_{y_i = k_m(x_i)} w_i^{(m)} e ^{-\alpha_i} + \sum_{y_i \ne k_m(x_i)} w_i^{(m)} e ^{\alpha_i} \\
&= \sum_{i = 1}^{N} w_i^{(m)} e ^{-\alpha_i} + \sum_{y_i \ne k_m(x_i)} w_i^{(m)} (e ^{\alpha_i} - e ^{- \alpha_i})
\end{align}

可知上式右边最后一项才与 $k_m$ 有关。所我们提高 $k_m$ 的正确率，就可减小最后一项：$\sum_{y_i \ne k_m(x_i)} w_i^{(m)} $，假设 $\alpha_m > 0$。 对损失误差函数求导其为 0 ，求出驻点就可以求得 $\alpha_m$.

即:

$$
\frac{dE}{d\alpha_m} = \frac{d\big( \sum_{y_i = k_m(x_i)} w_i^{(m)} e ^{-\alpha_i} + \sum_{y_i \ne k_m(x_i)} w_i^{(m)} e ^{\alpha_i} \big)}{d{\alpha_m}}
$$

最终求得驻点：

$$
\alpha_m = \frac{1}{2} ln\bigg( \frac{\sum_{y_i = k_m(x_i)} w_i^{(m)}}{\sum_{y_i \ne k_m(x_i)} w_i^{(m)}} \bigg)
$$

求驻点的证明可以参考 wiki.


假设 

$$ 
\epsilon_m = \frac{\sum_{y_i \ne k_m(x_i)} w_i^{(m)}}{\sum_{i=1}^{N} w_i^{(m)}} 
$$

则：

$$
\alpha_m = \frac{1}{2}ln\bigg(\frac{1 - \epsilon_m}{\epsilon_m}\bigg)
$$

我们 $k_m$ 弱分类器是已知的，求出了 $\alpha_m$ 后就可以都完成训练了。

### 算法

假设有：

- 训练集: $\{(x_1, y_1), \dots, (x_n, y_n)\}, y_i \in \{-1, + 1\}$
- 误差函数: $E(f(x), y, i) = e^{-y_i f(x_i)}$
- 弱分类器: $h: x -> \{-1, 1\}$
- 初始化权重: $w_{1, 1}, \cdots, w_{n 1} = \frac{1}{n}$

则进行迭代: $1 \dots T$

- 生成弱分类器 $h_t(X, y, W)$
    - 计算 $\epsilon_t$
    - 计算 $\alpha_t = \frac{1}{2} ln(\frac{1 - \epsilon_t}{\epsilon_t})$
- 添加集成分类器上
    - $F_t(x) = F_{t - 1}(x) + \alpha_t h_t (x)$
- 更新权重
    - $w_{i, t + 1} = w_{i, t} e ^{-y_i \alpha_t h_t(x_i)}, \forall i$
    - 重新规一化 $w_{i, t + 1}$, 使得 $\sum_{i}w_{i, t + 1} = 1$




### 『休斯』效应

在一些模型的应用中，特征维数的增加到某一临界点后，继续增加反而会导致分类器的性能变差。这种现象称之为 "Hugues Effect", 休斯效应或休斯现象。

AdaBoost 是可以解决休斯效应， The curse of dimensionality 。
